In [ ]:
!wget https://ai2-public-datasets.s3.amazonaws.com/arc/ARC-V1-Feb2018.zip

--2021-07-14 21:01:09--  https://ai2-public-datasets.s3.amazonaws.com/arc/ARC-V1-Feb2018.zip
Resolving ai2-public-datasets.s3.amazonaws.com (ai2-public-datasets.s3.amazonaws.com)... 52.218.154.43
Connecting to ai2-public-datasets.s3.amazonaws.com (ai2-public-datasets.s3.amazonaws.com)|52.218.154.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 680841265 (649M) [binary/octet-stream]
Saving to: ‘ARC-V1-Feb2018.zip’

ARC-V1-Feb2018.zip  100%[===================>] 649.30M  25.0MB/s    in 22s     

2021-07-14 21:01:31 (29.7 MB/s) - ‘ARC-V1-Feb2018.zip’ saved [680841265/680841265]



In [ ]:
!unzip ARC-V1-Feb2018.zip

Archive:  ARC-V1-Feb2018.zip
   creating: ARC-V1-Feb2018-2/
  inflating: ARC-V1-Feb2018-2/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/ARC-V1-Feb2018-2/
  inflating: __MACOSX/ARC-V1-Feb2018-2/._.DS_Store  
   creating: ARC-V1-Feb2018-2/ARC-Challenge/
  inflating: ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Dev.csv  
   creating: __MACOSX/ARC-V1-Feb2018-2/ARC-Challenge/
  inflating: __MACOSX/ARC-V1-Feb2018-2/ARC-Challenge/._ARC-Challenge-Dev.csv  
  inflating: ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Dev.jsonl  
  inflating: __MACOSX/ARC-V1-Feb2018-2/ARC-Challenge/._ARC-Challenge-Dev.jsonl  
  inflating: ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Test.csv  
  inflating: __MACOSX/ARC-V1-Feb2018-2/ARC-Challenge/._ARC-Challenge-Test.csv  
  inflating: ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Test.jsonl  
  inflating: __MACOSX/ARC-V1-Feb2018-2/ARC-Challenge/._ARC-Challenge-Test.jsonl  
  inflating: ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Train.csv  
  inflating: 

In [ ]:
!pip install farm-haystack

In [ ]:
!pip install spacy-entity-linker

     |████████████████████████████████| 6.4MB 6.8MB/s 
     |████████████████████████████████| 460kB 44.5MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 624kB 48.1MB/s 
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [ ]:
!python -m spacy_entity_linker "download_knowledge_base"

2021-07-14 21:05:30.153640: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
!python -m spacy download "en_core_web_md"

2021-07-14 21:06:05.404466: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 45.4MB 69kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
!pip install json-lines

In [ ]:
import spacy  # version 3.0.6'
import requests
from requests import utils
import pickle
import os
import json_lines
import random


from bs4 import BeautifulSoup
import requests

from haystack.preprocessor import PreProcessor


In [ ]:
def get_wikipedia_url_from_wikidata_id(wikidata_id, lang="en", debug=False):


    wikidata_id = "Q"+wikidata_id

    url = (
        'https://www.wikidata.org/w/api.php'
        '?action=wbgetentities'
        '&props=sitelinks/urls'
        f'&ids={wikidata_id}'
        '&format=json')
    json_response = requests.get(url).json()
    if debug: print(wikidata_id, url, json_response) 

    entities = json_response.get('entities')    
    if debug: print(entities) 
    if entities:
        entity = entities.get(wikidata_id)
        if debug: print(entity) 
        if entity:
            sitelinks = entity.get('sitelinks')
            if sitelinks:
                if lang:
                    # filter only the specified language
                    sitelink = sitelinks.get(f'{lang}wiki')
                    if sitelink:
                        wiki_url = sitelink.get('url')
                        if wiki_url:
                            return requests.utils.unquote(wiki_url)
                else:
                    # return all of the urls
                    wiki_urls = {}
                    for key, sitelink in sitelinks.items():
                        wiki_url = sitelink.get('url')
                        if wiki_url:
                            wiki_urls[key] = requests.utils.unquote(wiki_url)
                    return wiki_urls
    return None

In [ ]:
def turn_link_to_data(link):
    respond = requests.get(link)
    soup = BeautifulSoup(respond.text)
    l = soup.find_all('p')
    #for element in l:
    #  print(element.getText())
    to_index = []
    processor = PreProcessor(
        clean_empty_lines=True,
        clean_whitespace=True,
        clean_header_footer=True,
        split_by="word",
        split_length=100,
        split_respect_sentence_boundary=True,
        split_overlap=0
    )
    for element in l:
        data = {
                  'text': element.getText(),
                  'meta': {}
          }
        docs = processor.process(data)
        to_index = to_index + docs
        #print(docs)
    return to_index

In [ ]:
def raw_samples_to_links(samples):
    datas = []
    i = 0
    for sample in samples:
        i+=1
        if i%250 == 0:
            print(i)

        _id = sample["id"]
        _uniques = []
        _rscore = []
        _ret_data = []
        _article = []
        _options = []
        toomany = 0
            
        _answer = sample["answerKey"]
        
        if _answer == "5" or _answer == "E":
            toomany = random.randint(1, 4)   
            _answer = str(toomany)
            print(_id)
            print("Too many :")
            print(toomany)            
            
        if _answer == "1":
            _answer = "A"
        if _answer == "2":
            _answer = "B"
        if _answer == "3":
            _answer = "C"
        if _answer == "4":
            _answer = "D"        
        if _answer not in ["A", "B", "C", "D"]:
            print(_id)
            print("skipped")
            continue
        assert(_answer in ["A", "B", "C", "D"])

        _question = sample["question"]['stem']   
        for idx in range(len(sample['question']['choices'])): 
            answer = sample["question"]['choices'][idx]['label']
            if answer == "1":
                answer = "A"
            if answer == "2":
                answer = "B"
            if answer == "3":
                answer = "C"
            if answer == "4":
                answer = "D"
            
            if answer not in ["A", "B", "C", "D"]:
                continue
                
            if toomany > 0 and _answer == answer:
                _options.append(sample["question"]['choices'][4]['text'])
                continue                       
            
            _options.append(sample["question"]['choices'][idx]['text'])

        if len(_options) < 4:
            print(_id)
            print("skipped, less than 4 options")
            continue
        assert(len(_options) == 4)            
                
        _links = []
        doc = nlp(sample["question"]['stem'])
        all_linked_entities = doc._.linkedEntities
        for entity in all_linked_entities:
            ret = get_wikipedia_url_from_wikidata_id(str(entity.get_id()))
            if ret is not None:
                _links.append(ret)   
                _ret_data = _ret_data + turn_link_to_data(ret)
            else:
                parents = entity.get_super_entities(limit=1)
                for parent in parents:
                    ret = get_wikipedia_url_from_wikidata_id(str(parent.get_id())) 
                    if ret is not None:
                        _links.append(ret) 
                        _ret_data = _ret_data + turn_link_to_data(ret)  
                        break  

        ret_opt_data = []
        for option in _options:
            doc = nlp(option)
            all_linked_entities = doc._.linkedEntities  
            links = []              
            for entity in all_linked_entities:
                ret = get_wikipedia_url_from_wikidata_id(str(entity.get_id()))
                if ret is not None:
                    links.append(ret)   
                    ret_data = _ret_data + turn_link_to_data(ret)
                else:
                    parents = entity.get_super_entities(limit=1)
                    for parent in parents:
                        ret = get_wikipedia_url_from_wikidata_id(str(parent.get_id())) 
                        if ret is not None:
                            links.append(ret) 
                            ret_data = _ret_data + turn_link_to_data(ret)  
                            break  
                            
            ret_opt_data.append(ret_data)

        _ret_opt_data = ret_opt_data
        _opt_links = links

        data = data = {
                "id": _id,
                "article": [],
                "links": _links,
                "opt_links": _opt_links,
                "ret_data": _ret_data,
                "ret_opt_data": _ret_opt_data,
                "options": _options,
                "question": _question,
                "answer": _answer,
                "optionscore": [],
                "rscore": []
                }
        datas.append(data)
    return datas

In [ ]:
# initialize language model
nlp = spacy.load("en_core_web_md")

# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker", last=True)



datafile = "./ARC-V1-Feb2018-2/ARC-Easy/ARC-Easy-Test.jsonl"
filename = "_e_test_arc_links.pkl"

if os.path.isfile(filename):
    os.remove(filename)    
    

test_samples = []
with open(datafile) as f:
    for item in json_lines.reader(f):
        test_samples.append(item)



test = raw_samples_to_links(test_samples)
pickle.dump(test, open(filename, "wb"))   



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


07/14/2021 21:12:17 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 21:12:31 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 21:13:08 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 21:13:16 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 21:13:34 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 21:13:49 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 21:14:09 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 21:14:38 - WARNING - haystack.preprocessor.preproce

NYSEDREGENTS_2008_4_6
skipped, less than 4 options


07/14/2021 21:20:22 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 21:21:15 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 21:21:15 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 21:21:47 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 21:22:21 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 21:22:54 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 21:23:22 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 21:23:47 - WARNING - haystack.preprocessor.preproce

250


...

07/14/2021 23:33:50 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 23:35:11 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 23:35:15 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 23:35:28 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 23:35:28 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 23:35:32 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 23:35:32 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 23:35:35 - WARNING - haystack.preprocessor.preproce

NYSEDREGENTS_2010_4_19
skipped, less than 4 options
2000


07/14/2021 23:53:35 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 23:53:50 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 23:53:53 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 23:54:58 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 23:56:36 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 23:57:04 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 23:57:50 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/14/2021 23:58:13 - WARNING - haystack.preprocessor.preproce

2250


07/15/2021 00:12:53 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:12:54 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:12:55 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:13:14 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:13:24 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:13:36 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:13:38 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:13:42 - WARNING - haystack.preprocessor.preproce

In [ ]:
from google.colab import files
files.download( "_e_test_arc_links.pkl" )   

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# initialize language model
nlp = spacy.load("en_core_web_md")

# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker", last=True)



datafile = "./ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Test.jsonl"
filename = "_c_test_arc_links.pkl"

if os.path.isfile(filename):
    os.remove(filename)    
    

test_samples = []
with open(datafile) as f:
    for item in json_lines.reader(f):
        test_samples.append(item)



test = raw_samples_to_links(test_samples)
pickle.dump(test, open(filename, "wb"))   


07/15/2021 00:35:36 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:35:37 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:35:39 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:36:01 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:36:57 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:36:58 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:36:59 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:36:59 - WARNING - haystack.preprocessor.preproce

NYSEDREGENTS_2010_4_4
skipped, less than 4 options


07/15/2021 00:47:41 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:47:50 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:47:50 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:49:22 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:49:38 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:49:52 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:49:56 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:50:06 - WARNING - haystack.preprocessor.preproce

250


07/15/2021 00:58:21 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:59:42 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:59:45 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 00:59:50 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 01:00:01 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 01:00:10 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 01:00:47 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 01:00:48 - WARNING - haystack.preprocessor.preproce

...

07/15/2021 01:46:40 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 01:46:44 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 01:46:45 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 01:46:58 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 01:47:41 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 01:47:44 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 01:48:02 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 01:48:03 - WARNING - haystack.preprocessor.preproce

1000


07/15/2021 02:12:35 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 02:12:49 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 02:12:53 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 02:13:09 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 02:13:19 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 02:13:21 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 02:13:46 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 02:13:54 - WARNING - haystack.preprocessor.preproce

NYSEDREGENTS_2015_4_7
skipped, less than 4 options


07/15/2021 02:16:46 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 02:16:50 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 02:17:04 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 02:17:04 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 02:17:04 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 02:17:42 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 02:18:53 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
07/15/2021 02:18:53 - WARNING - haystack.preprocessor.preproce

In [ ]:
from google.colab import files
files.download( "_c_test_arc_links.pkl" )   

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with  open("_e_dev_arc_links.pkl", 'rb') as f:
    test_1 = pickle.load(f)

print(test_1[0])
print("-----------------------")
print(test_1[1])
print("-----------------------")
print(test_1[2])

{'id': 'MCAS_2000_4_6', 'article': [], 'links': ['https://en.wikipedia.org/wiki/Technology'], 'ret_data': [{'text': 'Technology ("science of craft", from Greek τέχνη, techne, "art, skill, cunning of hand"; and -λογία, -logia[2]) is the sum of techniques, skills, methods, and processes used in the production of goods or services or in the accomplishment of objectives, such as scientific investigation. Technology can be the knowledge of techniques, processes, and the like, or it can be embedded in machines to allow for operation without detailed knowledge of their workings. Systems (e.g.', 'meta': {'_split_id': 0}}, {'text': "machines) applying technology by taking an input, changing it according to the system's use, and then producing an outcome are referred to as technology systems or technological systems.", 'meta': {'_split_id': 1}}, {'text': 'The simplest form of technology is the development and use of basic tools. The prehistoric invention of shaped stone tools followed by the dis

In [ ]:
ret = get_wikipedia_url_from_wikidata_id("613299")
print(ret)

Q613299 https://www.wikidata.org/w/api.php?action=wbgetentities&props=sitelinks/urls&ids=Q613299&format=json {'entities': {'Q613299': {'type': 'item', 'id': 'Q613299', 'sitelinks': {'arwiki': {'site': 'arwiki', 'title': 'عبارة (علم الحاسوب)', 'badges': [], 'url': 'https://ar.wikipedia.org/wiki/%D8%B9%D8%A8%D8%A7%D8%B1%D8%A9_(%D8%B9%D9%84%D9%85_%D8%A7%D9%84%D8%AD%D8%A7%D8%B3%D9%88%D8%A8)'}, 'cswiki': {'site': 'cswiki', 'title': 'Příkaz (programování)', 'badges': [], 'url': 'https://cs.wikipedia.org/wiki/P%C5%99%C3%ADkaz_(programov%C3%A1n%C3%AD)'}, 'dawiki': {'site': 'dawiki', 'title': 'Sætning (programmering)', 'badges': [], 'url': 'https://da.wikipedia.org/wiki/S%C3%A6tning_(programmering)'}, 'dewiki': {'site': 'dewiki', 'title': 'Anweisung (Programmierung)', 'badges': [], 'url': 'https://de.wikipedia.org/wiki/Anweisung_(Programmierung)'}, 'enwiki': {'site': 'enwiki', 'title': 'Statement (computer science)', 'badges': [], 'url': 'https://en.wikipedia.org/wiki/Statement_(computer_scienc

In [ ]:
respond = requests.get("https://en.wikipedia.org/wiki/Statement_(computer_science)")
soup = BeautifulSoup(respond.text)
l = soup.find_all('p')
for element in l:
  print(element.getText())

processor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
    split_overlap=0
)
for element in l:
  data = {
            'text': element.getText(),
            'meta': {}
    }
  docs = processor.process(data)
  print(docs)

In computer programming, a statement is a syntactic unit of an imperative programming language that expresses some action to be carried out.[1] A program written in such a language is formed by a sequence of one or more statements. A statement may have internal components (e.g., expressions).

Many programming languages (e.g. Ada, Algol 60, C, Java, Pascal) make a distinction between statements and definitions/declarations. A definition or declaration specifies the data on which a program is to operate, while a statement specifies the actions to be taken with that data.

Statements which cannot contain other statements are simple; those which can contain other statements are compound.[2]

The appearance of a statement (and indeed a program) is determined by its syntax or grammar. The meaning of a statement is determined by its semantics.

Simple statements are complete in themselves; these include assignments, subroutine calls, and a few statements which may significantly affect the pr

In [ ]:

# initialize language model
nlp = spacy.load("en_core_web_md")

# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker", last=True)

doc = nlp("During an investigation, heat transferred from a liquid to the environment. Which pair of explanations can best account for this result?")

# returns all entities in the whole document
all_linked_entities = doc._.linkedEntities
# iterates over sentences and prints linked entities
#for sent in doc.sents:    
#    sent._.linkedEntities.pretty_print()

for entity in all_linked_entities:
    print(entity.get_label())

https://www.wikidata.org/wiki/Q44432      44432      heat                            energy transfer process, or its amount (and direction), that is associated with a temperature differ
https://www.wikidata.org/wiki/Q21004260   21004260   inquiry                         process that has the aim of augmenting knowledge, resolving doubt, or solving a problem             
https://www.wikidata.org/wiki/Q11435      11435      liquid                          liquid object                                                                                       
https://www.wikidata.org/wiki/Q2249676    2249676    environment                     surrounding of an organism or population                                                            
https://www.wikidata.org/wiki/Q191290     191290     ordered pair                    pair of mathematical objects; tuple of specific length (tuple length n=2)                           
https://www.wikidata.org/wiki/Q7958       7958       explanation      